In [32]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [ ]:
data = pd.read_csv('apartments_for_rent_classified_100K.csv', sep=";", encoding='cp1252')

In [ ]:
# Handle missing values
imputer = SimpleImputer(strategy='mean')
data['bathrooms'] = imputer.fit_transform(data[['bathrooms']])
data['bedrooms'] = imputer.fit_transform(data[['bedrooms']])
data['square_feet'] = imputer.fit_transform(data[['square_feet']])
#ortalamanın değerini verir
#remove time id and adress columns
data.drop(['id', 'title', 'body', 'address', 'time','price_display'], axis=1)


In [ ]:
amenities = data['amenities'].str.get_dummies(sep=',')
data = pd.concat([data, amenities], axis=1)
data = data.drop(columns=['amenities'])
data



In [ ]:
corr = data[['price', 'latitude', 'longitude', 'square_feet', 'bedrooms', 'bathrooms', 'Patio/Deck', 'Playground', 'Pool', 'Refrigerator', 'Storage', 'TV', 'Tennis', 'View', 'Washer Dryer', 'Wood Floors']].corr()
corr
fig, ax = plt.subplots(figsize=(15,15))         # Sample figsize in inches
#heatmap with small text
sns.heatmap(corr, annot=True,linewidths=.5, ax=ax)

In [ ]:
#remove category column
data = data.drop(columns=['title','body','time','price_display','currency',])



In [ ]:
X = data.drop(['price'], axis=1)
y = data['price']

# Scale the numerical features
scaler = StandardScaler()
X = scaler.fit_transform(X)
